In [14]:
import ast
from collections import Counter
import math
import zlib
import pandas as pd
import numpy as np 
import mido
from collections import defaultdict

In [15]:
output_directory = "music_metrics.csv"

In [16]:
def extract_features(notes_dict):
    pitches = [note[0] for note in notes_dict.values()]
    durations = [note[1] for note in notes_dict.values()]
    velocities = [note[2] for note in notes_dict.values()]
    return pitches, durations, velocities
def calculate_entropy(sequence):
    counts = Counter(sequence)
    probabilities = [count / len(sequence) for count in counts.values()]
    return -sum(p * math.log2(p) for p in probabilities if p > 0)
def calculate_compression_ratio(sequence):
    # Convert the sequence to a string for compression
    sequence_str = ",".join(map(str, sequence))
    compressed = zlib.compress(sequence_str.encode('utf-8'))
    return len(compressed) / len(sequence_str.encode('utf-8'))

def calculate_metrix(df):
    complexity_scores = []
    for _, row in df.iterrows():
        notes_dict = ast.literal_eval(row["Notes"])  # Convert string back to dictionary
        pitches, durations, velocities = extract_features(notes_dict)

        metrics = {
        "pitch_entropy": calculate_entropy(pitches),
        "duration_entropy": calculate_entropy(durations),
        "velocity_entropy": calculate_entropy(velocities),
        "pitch_compression": calculate_compression_ratio(pitches),
        "duration_compression": calculate_compression_ratio(durations),
        "velocity_compression": calculate_compression_ratio(velocities),
        }

        complexity_scores.append(metrics)

    # Add the complexity column to the dataframe
    return complexity_scores

In [17]:
df = pd.read_csv(output_directory)
df["Metrix"] = calculate_metrix(df)
print(df["Metrix"])

Series([], Name: Metrix, dtype: float64)


In [18]:
df.shape

(0, 7)

In [19]:
def extract_notes_from_df(df):
    # Crée trois nouvelles colonnes pour les pitches, durations et velocities
    df["pitches"] = df["Notes"].apply(lambda x: [note[0] for note in ast.literal_eval(x).values()])
    df["durations"] = df["Notes"].apply(lambda x: [note[1] for note in ast.literal_eval(x).values()])
    df["velocities"] = df["Notes"].apply(lambda x: [note[2] for note in ast.literal_eval(x).values()])
    return df


df = extract_notes_from_df(df)

In [20]:
#l = [df["pitches"][0],df["durations"][0],df["velocities"][0]]


print(df["pitches"][0][0]) #df["pitches"] est en 2D

KeyError: 0

Il n'y a que des partitions de piano dans le dataset, une seule piste à chaque morceau/ligne (donc ce qu'on voulait nice)

In [ ]:
#Je veux des listes des triplets 


def extract_triplets(df):
    """
    Convert the 'Notes' column in the DataFrame into a list of triplets (pitches, durations, velocities).
    
    Args:
        df (pd.DataFrame): DataFrame containing a 'Notes' column with dictionary-like strings.
        
    Returns:
        pd.DataFrame: Updated DataFrame with a new column 'triplets' containing the list of tuples.
    """
    df["triplets"] = df["Notes"].apply(lambda x: [tuple(note) for note in ast.literal_eval(x).values()])
    return df

In [ ]:
df = extract_triplets(df)

**On encode notre musique en une séquence de mots, écrits en bits. Les mots sont séparés par des espaces.**

Séquence : position - p - d - v


Approche de la complexité de Kolmogorov
En gros une note c'est un triplet (p,d,v) (pitch, duratio,velocity)
Donc deux notes identiques c'est deux notes qui ont les mêmes p,d,v
Si on a deux mêmes p,d,v alors on a juste à écrire la note une fois, et à mettre ses 2 emplacements. C'est moins long que d'encoder deux fois l'emplacement et le p,d,v. Donc premier facteur de réduction de complexité


Savoir qu'il y a plusieurs notes de même p ça sert pas forcément à grand chose car il faut quand même noter leur emplacement
Mais quand plusieurs notes de même p ou d ou v se suivent la c'est intéressant et ça fait diminuer la complexité
Donc quand on a des notes qui se suivent avec mêmes p ou d ou v, complexité diminue

pareil si c'est pour des couples de var : si deux notes se suivent avec même p,v (mais d différent) alors ça fait quand même bien diminuer la complexité 
Algo : 

1) on imagine que notre code code toutes les notes uniques et sans aucunes répetitions de p,d,v à droite ou à gauche, avec leurs emplacements

2) puis il prend celles qui se répètentsur une ou 2 vars, et les codes ensemble

3) puis il prend les notes identiques et code juste leur emplacement

Si une note est à la fois exactement égale à une autre, et apparait dans une répétition de p, alors comment on l'encode ? Ca c'est une bonne question (elle appartient aux notes de catégorie 2 et 3 à la fois)
-> Pour l'instant on considère que la méthode 3 est toujours plus efficace, donc on classe ce genre de note dans la classe 3

--> En vérité ça va dépendre de la longueur de la répétition d'une var. Si ça fait beaucoup diminuer la complexité des notes voisines alors il faudrait prendre encodage 2. Mais y a aussi un problème avec cette espèce de "valeur" de complexité. On fait des +1 -1 genre ?

Pour l'instant oui 

Plus long encodage : chaque triplet et son emplacement 

Taille de l'encodage d'un triplet (𝑝,𝑑,𝑣) : constante 𝐶_note
Taille de l'encodage de la position : constant 𝐶_position
nombre de notes : n 
Taille maximale de l'encodage : n x (𝐶_note + 𝐶_position) 

In [ ]:
mean_length = np.mean([len(df["triplets"][i]) for i in range(len(df["triplets"]))])
print(mean_length)

max_length = np.max([len(df["triplets"][i]) for i in range(len(df["triplets"]))])
print(max_length)

1108.610510805501
9824


taille encodage p : p varie de 0 à 127, il est donc écrit sur 7 bits

d en millisecondes, ne dépasse pas 10 000 ms :  14 bits

v varie de 0 à 127 : 7 bits

𝐶_note = 28 bits

la partition la plus longue comporte : 9824 notes
9824 se code sur 14 bits
donc la position se code sur 14 bits

𝐶_position = 14 bits

Taille maximale de l'encodage = n x 42 bits

Séquence :  p - d - v - position
Soit :      7   14  7      14       en bits

Si on voit plusieurs séquences de 14 bits d'affilées, c'est qu'on a un triplet identique à plusieurs endroits, et qu'on encode à la suite les positions

(peut-être que ce serait bien de préciser la position que dans ce cas d'ailleurs, car sinon la suite se lit comme une séquence, les notes sont codées les unes après les autres selon leur ordre d'apparition, donc on connait déjà leur position)

il faut avoir une manière de préciser dans le code qu’on a une répétition et la taille de la répétition. Il y a 5 types de répétitions possibles : d, p , v ou de doublets (p,v) , (p,d) (d,v), ça se code sur 4 bits. Ca tombe bien rien ne fait 4 bits dans ce qui précéde, donc dans notre code cès qu'on voit 4 bits, ça veut dire qu'il y a répétition. Ca dit qu'est-ce qui est répété. Puis ensuite ça donne le nombre de répétitions. Pas sur que ce 'quand tu vois un mot de 4 bits c'est que c'est pour préciser une répétition donc tu sais ce que ça veut dire' soit très correct. On suppose que les mots sont séparés par des espaces. Est-ce qu'on peut faire cette hypothèse ? 

In [ ]:
partition = df["triplets"][0]

In [ ]:
#print(partition)

In [ ]:
def detect_continuous_repetitions(triplets):
    """
    Détecte les répétitions contiguës dans une liste de triplets (p, d, v) 
    et les classe par type de répétition.
    
    Args:
        triplets (list of tuples): Liste des notes sous forme de triplets (p, d, v).
        
    Returns:
        dict: Dictionnaire contenant les répétitions détectées sous forme de listes d'indices.
    """
    repetitions = {
        "p": [],       # Répétitions de pitches
        "d": [],       # Répétitions de durations
        "v": [],       # Répétitions de velocities
        "p_v": [],     # Répétitions de (p, v)
        "p_d": [],     # Répétitions de (p, d)
        "d_v": []      # Répétitions de (d, v)
    }
    
    n = len(triplets)
    
    # Fonction pour détecter les répétitions sur une clé donnée
    def detect_repetition(key_func):
        result = []
        start = 0  # Début d'une répétition
        for i in range(1, n):
            if key_func(triplets[i]) == key_func(triplets[i - 1]):
                continue  # La répétition continue
            else:
                # Fin de la répétition
                if i - start > 1:  # Une répétition doit avoir au moins 2 éléments
                    result.append(list(range(start, i)))
                start = i  # Nouvelle répétition
        # Ajouter la dernière répétition
        if n - start > 1:
            result.append(list(range(start, n)))
        return result
    
    # Appliquer la détection pour chaque type
    repetitions["p"] = detect_repetition(lambda t: t[0])  # Répétitions sur p
    repetitions["d"] = detect_repetition(lambda t: t[1])  # Répétitions sur d
    repetitions["v"] = detect_repetition(lambda t: t[2])  # Répétitions sur v
    repetitions["p_v"] = detect_repetition(lambda t: (t[0], t[2]))  # Répétitions sur (p, v)
    repetitions["p_d"] = detect_repetition(lambda t: (t[0], t[1]))  # Répétitions sur (p, d)
    repetitions["d_v"] = detect_repetition(lambda t: (t[1], t[2]))  # Répétitions sur (d, v)
    
    return repetitions

triplets = [(55, 305, 84), (55, 305, 84),(74, 600, 120),(55, 305, 84),(55, 305, 84),(55, 305, 84)]

dico_repet = detect_continuous_repetitions(triplets)

print(dico_repet)

{'p': [[0, 1], [3, 4, 5]], 'd': [[0, 1], [3, 4, 5]], 'v': [[0, 1], [3, 4, 5]], 'p_v': [[0, 1], [3, 4, 5]], 'p_d': [[0, 1], [3, 4, 5]], 'd_v': [[0, 1], [3, 4, 5]]}


In [ ]:
def detect_prioritized_repetitions(partition):
    """
    Détecte les répétitions contiguës de doublets (p, d), (p, v), (d, v) 
    et les répétitions simples dans une liste de partition.
    """
    repetitions = {
        "p_d": [],
        "d_v": [],
        "p_v": [],
        "p": [],
        "d": [],
        "v": []
    }

    n = len(partition)
    used_indices = set()

    def detect_repetition(key_func, existing_indices):
        result = []
        start = 0
        for i in range(1, n):
            if key_func(partition[i]) == key_func(partition[i - 1]) and i not in existing_indices:
                continue
            else:
                if i - start > 1 and not any(j in existing_indices for j in range(start, i)):
                    result.append(list(range(start, i)))
                start = i
        if n - start > 1 and not any(j in existing_indices for j in range(start, n)):
            result.append(list(range(start, n)))
        return result

    # Détection des doublets
    repetitions["p_d"] = detect_repetition(lambda t: (t[0], t[1]), used_indices)
    used_indices.update([idx for group in repetitions["p_d"] for idx in group])

    repetitions["d_v"] = detect_repetition(lambda t: (t[1], t[2]), used_indices)
    used_indices.update([idx for group in repetitions["d_v"] for idx in group])

    repetitions["p_v"] = detect_repetition(lambda t: (t[0], t[2]), used_indices)
    used_indices.update([idx for group in repetitions["p_v"] for idx in group])

    # Détection des répétitions simples
    repetitions["p"] = detect_repetition(lambda t: t[0], used_indices)
    used_indices.update([idx for group in repetitions["p"] for idx in group])

    repetitions["d"] = detect_repetition(lambda t: t[1], used_indices)
    used_indices.update([idx for group in repetitions["d"] for idx in group])

    repetitions["v"] = detect_repetition(lambda t: t[2], used_indices)

    return repetitions

#TEST

partition = [(56, 305, 85), (55, 305, 85),(74, 600, 120),(53, 305, 85),(54, 305, 85),(51, 305, 85)]

dico_repet = detect_prioritized_repetitions(partition)

print(dico_repet)

{'p_d': [], 'd_v': [[0, 1], [3, 4, 5]], 'p_v': [], 'p': [], 'd': [], 'v': []}


In [ ]:
from collections import Counter

def traiter_same_notes(partition):
    """
    Détecte les triplets présents au moins deux fois dans la liste,
    renvoie un dictionnaire avec leur nombre d'occurrences
    et une liste sans aucune occurrence des triplets répétés.

    Args:
        partition (list of tuples): Liste des triplets (p, d, v).

    Returns:
        dict: Dictionnaire avec les triplets répétés comme clés et leur occurrence comme valeurs.
        list: Liste des triplets sans aucune occurrence des répétitions.
    """
    # Etape 1 : créer la liste des triplets 
    liste_triplets = [quadruplet[:3] for quadruplet in partition]

    # Étape 1 : Compter les occurrences de chaque triplet

    compteur = Counter(liste_triplets)
    
    # Étape 2 : Construire un dictionnaire des triplets ayant des occurrences >= 2
    dico_repetitions = {triplet: count for triplet, count in compteur.items() if count >= 2}
    
    # Étape 3 : Construire une nouvelle liste en excluant les triplets dupliqués
    nouvelle_partition = [quadruplet for quadruplet in partition if quadruplet[:3] not in dico_repetitions]
    
    return dico_repetitions, nouvelle_partition



partition = [(55, 305, 85,1), (55, 305, 85,2),(74, 600, 120,4),(55, 305, 85,8),(55, 305, 85,12),(51, 305, 85,42)]

dico,liste = traiter_same_notes(partition)

print(liste)

[(74, 600, 120, 4), (51, 305, 85, 42)]


In [ ]:
#Code harmonique
#On doit le faire avant le reste

def algo_harmoniques(partition) :
    # Initialisation
    dico_accords = {}  # Dictionnaire pour stocker les accords et leur valeur associée
    valeur_incr = 128  # Valeur de départ pour les accords
    partition_accords = []  # Liste pour stocker la nouvelle partition avec accords
    gain_total = 0  # Variable pour stocker le gain total

    # Regrouper les quadruplets par start_time
    accords_par_temps = defaultdict(list)
    for quadruplet in partition:
        pitch, duration, velocity, start_time = quadruplet
        accords_par_temps[start_time].append(quadruplet)

    # Parcourir les accords regroupés
    for start_time, notes in accords_par_temps.items():
        if len(notes) > 1:  # Il s'agit d'un accord si plusieurs notes partagent le même start_time
            # Créer un tuple représentant l'accord (les pitches triés pour garantir l'unicité)
            pitches = tuple(sorted(note[0] for note in notes))

            # Ajouter l'accord au dictionnaire s'il n'existe pas
            if pitches not in dico_accords:
                dico_accords[pitches] = valeur_incr
                valeur_incr += 1

            # Calculer le gain pour cet accord
            gain = 20 * (len(notes) - 1)
            gain_total += gain

            # Créer un seul quadruplet pour l'accord
            duration = notes[0][1]  # Prendre la duration de la première note
            velocity = notes[0][2]  # Prendre la velocity de la première note
            partition_accords.append((dico_accords[pitches], duration, velocity, start_time))
        else:
            # Garder les notes seules inchangées
            partition_accords.append(notes[0])

    # Remplacer la partition originale
    partition = partition_accords

    return partition,gain_total

In [ ]:
def estim_complexite(partition) :
    """ partition : liste de tuples de 3 int. Chaque tuple représente une note
    Cette fonction estime la complexité de la partition, à partir des hypothèses émises précédemment
    """
    worst_complexity = 42*len(partition) #Comme expliqué plus haut 
    #On commence par les accords
    partition,gain_accords = algo_harmoniques(partition)
    #détecter les triplets identiques
    same_triplets,partition_reduite = traiter_same_notes(partition)
    #On utilisera le dictionnaire same_triplets, pour calculer la réduction de complexité, après

    #détecter les répétitions de d, p , v ou de doublets (p,v),(p,d) (d,v)
    repetitions = detect_prioritized_repetitions(partition_reduite)

    #Maintenant on va utiliser nos deux dictionnaires 'same_triplets' et repetitions pour calculer la réduction de complexité

    #On commence par les triplets identiques 
    gain_triplets_identiques = sum((count - 1) * 28 for count in same_triplets.values())

    #Maintenant gain sur les répétitions de doublet ou simple variable
    # Définition des économies par type
    gains_par_type = {
        "p": 7,
        "d": 14,
        "v": 7,
        "p_d": 21,
        "p_v": 14,
        "d_v": 21
    }
    
    gain_repetitions = 0
    
    # Boucle sur chaque type de répétition
    for type_repet, repetitions in repetitions.items():
        for repetition in repetitions:
            taille = len(repetition)  # Taille de la répétition
            if taille > 1:  # Les répétitions doivent être d'au moins 2 notes
                # Calcul des bits économisés pour cette répétition
                bits_economises = (taille - 1) * gains_par_type[type_repet]
                
                # Bits pour encoder le type (4 bits) et la taille (log2(taille))
                bits_ajoutes = 4 + math.ceil(math.log2(taille))
                
                # Gain net
                gain_net = bits_economises - bits_ajoutes
                gain_repetitions += gain_net

    
    #Et le calcul final
    Complexity = worst_complexity - gain_triplets_identiques - gain_repetitions - gain_accords

    return Complexity

**Explication des calculs**
**Pour les triplets identiques :** 

On doit coder le triplet une seule fois, puis les positions. La complexité est alors de :
C = 28 + 14 x nb_repetition_du_triplet. On économise 28 bits à chaque fois qu'on a un triplet identique en +. On encode 1 fois pour tous les nb_repetition_du_triplet de la partition. Donc on gagne 28x(nb_repetition_du_triplet - 1). Pour chaque triplet répété.

**Pour les répétitions de doublets ou simples variables :**

On doit coder de quel type de répétition il s'agit. 6 types de répétition possibles, donc cette info se code sur 4 bits. C'est d'ailleurs le seul mot de 4 bits possible dans notre séquence, donc quand on voit un mot de 4 bits on sait qu'on a à faire à une répétition. Il faut aussi encoder la taille de la répétition : sur combien de notes la répétition a lieu. Cette valeur vaut log2(taille_répétition). En fonction de la ou les variables qui se répètent on économise plus ou moins de bits. p fait 7 bits donc les répétitions de p économisent 7 bits à chaque fois (moins tous ceux qu'on a du ajouter pour encoder la répétition). Pour une répétition de d on économise 14 bits à chaque fois. Pour des doublets on économise sur chaque variable. Si on a une répétion d_p, on économise 7 + 14 = 21 bits par répétition. On encode donc cela.

In [ ]:
#Tests
partition_test = df["triplets"][0]


print("Complexité estimée",estim_complexite(partition_test))
print("worst_case complexity = ",42*len(partition_test))

print("ratio de compression via ma méthode = ",(estim_complexite(partition_test))/(42*len(partition_test)))




NameError: name 'df' is not defined

In [ ]:
#Calcul de la compression zip

def calculer_ratio_compression(partition):
    """
    Calcule le ratio de compression d'une partition de triplets en utilisant zlib.

    Args:
        partition (list of tuples): Liste de triplets (p, d, v).

    Returns:
        float: Ratio de compression (valeur entre 0 et 1, où plus proche de 0 = meilleure compression).
    """
    # Étape 1 : Convertir la partition en chaîne de caractères
    partition_str = ",".join(f"{t[0]}_{t[1]}_{t[2]}" for t in partition)
    
    # Étape 2 : Compresser la chaîne avec zlib
    partition_bytes = partition_str.encode('utf-8')  # Conversion en bytes
    compressed_bytes = zlib.compress(partition_bytes)
    
    # Étape 3 : Calculer le ratio de compression
    ratio_compression = len(compressed_bytes) / len(partition_bytes)
    
    return ratio_compression

In [ ]:
#Tests
partition_test = df["triplets"][0]


print("Complexité estimée",estim_complexite(partition_test))
print("worst_case complexity = ",42*len(partition_test))

print("ratio de compression via ma méthode = ",(estim_complexite(partition_test))/(42*len(partition_test)))
print("ratio de compression avec zip = ",calculer_ratio_compression(partition_test))

Complexité estimée 27298
worst_case complexity =  55944
ratio de compression via ma méthode =  0.48795223795223797
ratio de compression avec zip =  0.08481277040131284


In [ ]:
#print(df["Notes"][0])

In [ ]:
import os
from mido import MidiFile

# Fonction pour extraire les notes au format (p, d, v, t)
def extract_note_data(midi_path):
    note_data = []
    midi = MidiFile(midi_path)
    current_time = 0  # Temps accumulé en ticks
    for track in midi.tracks:
        for msg in track:
            if not msg.is_meta:
                current_time += msg.time  # Temps accumulé
                if msg.type == 'note_on' and msg.velocity > 0:
                    note_data.append((msg.note, 0, msg.velocity, current_time))
    return note_data

# Parcourir tous les fichiers MIDI
extract_path = "adl-piano-midi"
all_notes = []

for root, _, files in os.walk(extract_path):
    for file in files:
        if file.endswith(".mid") or file.endswith(".midi"):
            midi_path = os.path.join(root, file)
            notes = extract_note_data(midi_path)
            all_notes.extend([(file, note) for note in notes])

# Afficher un aperçu des notes extraites
print("Exemple de notes extraites :")
print(all_notes[:10])


**Problème : on a en réalité 4 variables, donc on doit refaire un dataset accurate. On essaie dans un nouveau Notebook**

partition : c'est une liste de triplets 

On s'en fout de la nature du triplet, tant qu'on a 3 valeurs

On va juste convertir les bonnes données, en triplet. Et voila 

Pour les harmoniques, quand y'a un delta_t à 0 ça veut dire 2 notes sont jouées en même temps. Si ça correspond à un accord, alors on encode le nom de l'accord à la place des deux notes.